In [1]:
%run variables.ipynb

In [2]:
import pandas as pd
import numpy as np
import os


def generate_database(path, db_paths, scimago_codes):
    
    db = {}
    df = pd.DataFrame()

    ## Loading pre-treated databases; the parts which these are needed are labelled with this same indentation
    df_pop = pd.read_csv(db_paths['total_population_path'], index_col=0 )
    df_pop_code = pd.read_csv(db_paths['over_25_population_path'], index_col=0 )
    df_pop_code.rename(columns={'Total': 'Population Total', 'Numeric code': 'Country code'}, inplace=True)
    df_schooling_new = pd.read_csv(db_paths['schooling_wb-gdp'], index_col=0 )
    df_country_codes = pd.read_csv(db_paths['country_info_path'], index_col=0 )
    df_gdp = pd.read_csv(db_paths['gdp_pwt'], index_col=0 )
    df_gov = pd.read_csv(db_paths['gov_effectiveness_path'], index_col=0 )
    df_sci_tech = pd.read_csv(db_paths['sci_tech_path'], index_col=0 )


    for subdir, dirs, files in os.walk(path):
        # print (subdir)
        # print (dirs)
        # print (files)

        if len(files) > 0:

            subject = subdir[17:]
            print (f"Currently compiling data for {subject}...\n")

            db[subject] = {}

            for filename in files:
                filepath = subdir + os.sep + filename

                if filepath.endswith(".xlsx"):
                    print (f"\tGenerating dataframe for file '{filepath}'\n")

    ## This next part requires df_pop

                    df_scimago = pd.read_excel(filepath)
                    year = int(filepath[-9:-5])
                    country_codes = []

                    for index, row in df_scimago.iterrows():
                        if (row['Country'] == df_pop.iloc[:, 1]).any():
                            filter = (df_pop.iloc[:, 1] == row['Country']) & (df_pop['Year'] == year)
                            country_codes.append(df_pop.loc[filter, "Country code"].item())
                        else:
                            # print(f"\tCountry code for {row['Country']} not found in population database, using {scimago_codes[row['Country']]} instead")
                            country_codes.append(scimago_codes[row['Country']])

                    df_scimago["Country code"] = pd.Series(country_codes)
                    h_index = pd.DataFrame(df_scimago.set_index('Country code').loc[:, "H index"])
                    h_index = h_index.groupby(h_index.index).max()

                    ## To see countries grouped by their sovereign country, add df_scimago['Country'] as argument in the list for groupby()
                    df_scimago = df_scimago[['Documents', 'Citable documents', 'Citations', 'Self-citations']].groupby([df_scimago['Country code']]).sum()
                    
                    ## creating Citations per Document column and appending former H_index column
                    df_scimago['Citations per Document'] = round((df_scimago['Citations'] / df_scimago['Documents']), 2)
                    df_scimago = df_scimago.join(h_index, how='left')

                    ## adding subject column
                    df_scimago['Subject'] = subject

                    ## df_pop must be reindexed for this next part; year filter is applied  
                    df_pop_indexed = df_pop.set_index('Country code')
                    df_pop_new = df_pop_indexed[df_pop_indexed['Year'] == year]
                    df_pop_new['Population 25 Years Before'] = df_pop_indexed[df_pop_indexed['Year'] == (year - 25)]['Population']
                    
                    ## Netherlands Antilles is composed mainly from the population of 4 islands, the main ones being Curaçao and Aruba
                    if (df_scimago.index.get_level_values(0) == 530).any():
                        pop_netherlands_antilles = round(df_pop_new[(df_pop_new.index == 531) | (df_pop_new.index == 534) | (df_pop_new.index == 533) | (df_pop_new.index == 535)][['Population', 'Population 25 Years Before']].apply('sum'), 3)
                        df_pop_new.loc[530] = ["Netherlands Antilles", year, pop_netherlands_antilles[0], pop_netherlands_antilles[1]]

    ## This next part requires df_pop_code

                    ## df_pop_code must be reindexed for this next part; year filter is applied  
                    df_pop_code_new = df_pop_code.set_index('Country code')
                    df_pop_code_new = df_pop_code_new[df_pop_code_new['Year'] == year].iloc[:,[3,4]]
                    
                    ## merge between df_pop and df_pop_code
                    df_pop_new = df_pop_new.join(df_pop_code_new, how='left')

                    ## merge between df_pop_new and df_scimago
                    df_scimago_pop = df_pop_new.join(df_scimago, how='right')

                    ## changing type from string to float
                    df_scimago_pop[['Population', 'Population 25 Years Before']].astype('float')

    ## This next part requires df_schooling_new

                    ## Filering by year and retrieving target columns
                    df_schooling = df_schooling_new.loc[(df_schooling_new['Year'] == year), ['Avg. Adult Schooling','GDP per capita (World Bank)']]
                    
                    ## merge between df_schooling and df_scimago-pop
                    df_sci_pop_schooling = df_scimago_pop.join(df_schooling, how='left')

    ## This next part requires df_country_codes
                    
                    ## retrieving target columns from df_country_codes
                    df_country_independence = pd.DataFrame(df_country_codes.set_index('Numeric code').loc[:, ['Independent', 'Sink OFC', 'Conduit OFC', 'Code']])
                    
                    ## merge between df_country_codes and df_sci_pop_schooling
                    df_sci_pop_schooling = df_country_independence.join(df_sci_pop_schooling, how='right')
                    
    ## This next part requires df_gdp

                    ## filtering by year and retrieving target column
                    filter = (df_gdp['Year'] == year)
                    df_gdp_new = df_gdp.loc[filter, 'GDP per capita (PWT)']

                    ## merging df_gdp_new with df_sci_pop_schooling
                    df_sci_pop_schooling = df_sci_pop_schooling.join(df_gdp_new, how='left')

                    ## adding stats my own stats
                    df_sci_pop_schooling['MAX TAMS'] = df_sci_pop_schooling['Avg. Adult Schooling'] * df_sci_pop_schooling['Population 25 Years Before']
                    df_sci_pop_schooling['REAL TAMS'] = df_sci_pop_schooling['Avg. Adult Schooling'] * df_sci_pop_schooling['Population 25+']
                    df_sci_pop_schooling['Documents / M-TAMS'] = df_sci_pop_schooling['Documents'] / df_sci_pop_schooling['MAX TAMS']
                    df_sci_pop_schooling['Documents / R-TAMS'] = df_sci_pop_schooling['Documents'] / df_sci_pop_schooling['REAL TAMS']
                    df_sci_pop_schooling['Citations / M-TAMS'] = df_sci_pop_schooling['Citations per Document'] * df_sci_pop_schooling['Documents / M-TAMS']
                    df_sci_pop_schooling['Citations / R-TAMS'] = df_sci_pop_schooling['Citations per Document'] * df_sci_pop_schooling['Documents / R-TAMS']

                    ## deleting Population Total column; this data is repeated in Population column (rounded to 2 decimals)
                    del df_sci_pop_schooling['Population Total']

    ## This next part requires df_gov

                    ## filtering by year
                    df_gov_new = df_gov[df_gov['Year'] == year]

                    ## merging df_gov with df_sci_pop_schooling
                    df_sci_pop_schooling_gov = df_sci_pop_schooling.join(pd.DataFrame(df_gov_new['Gov. Effectiveness']), how='left')
                    df_sci_pop_schooling_gov.sort_values(by=['Year', 'Country'], inplace=True)
                    
    ## This next part requires df_sci_tech

                    ## filtering by year and removing Year column
                    df_sci_tech_new = df_sci_tech[df_sci_tech['Year'] == year].iloc[:, 1:]

                    ## merging df_gov with df_sci_pop_schooling
                    df_sci_tech_pop_schooling_gov = df_sci_pop_schooling_gov.join(df_sci_tech_new, how='left')
    
    ## Appending data to df and db

                    print (f'\t\tAppending dataframe for {year} to main dataframe...')
                    df = df.append(df_sci_tech_pop_schooling_gov)
                    
                    print (f'\t\tCreating dictionary for {year} and appending to database...')
                    df_dict = df_sci_tech_pop_schooling_gov.to_dict()
                    db[subject][year]= df_dict

                    print (f'\t\tDone!\n')
                    
                ## uncomment the lines below to test generation of database for just the first year of every subject
                #     print(df_sci_tech_pop_schooling_gov)
                # break
    
    print (f'Saving main dataframe to db.csv file')
    df.to_csv('../outputs/db.csv')
    print ('Done!')
    
    return [df, db]

In [3]:
df, db = generate_database('..\subject_areas', db_paths, scimago_codes)

ing dictionary for 2001 and appending to database...
		Done!

	Generating dataframe for file '..\subject_areas\Physics and Astronomy\scimagojr country rank 2002.xlsx'

		Appending dataframe for 2002 to main dataframe...
		Creating dictionary for 2002 and appending to database...
		Done!

	Generating dataframe for file '..\subject_areas\Physics and Astronomy\scimagojr country rank 2003.xlsx'

		Appending dataframe for 2003 to main dataframe...
		Creating dictionary for 2003 and appending to database...
		Done!

	Generating dataframe for file '..\subject_areas\Physics and Astronomy\scimagojr country rank 2004.xlsx'

		Appending dataframe for 2004 to main dataframe...
		Creating dictionary for 2004 and appending to database...
		Done!

	Generating dataframe for file '..\subject_areas\Physics and Astronomy\scimagojr country rank 2005.xlsx'

		Appending dataframe for 2005 to main dataframe...
		Creating dictionary for 2005 and appending to database...
		Done!

	Generating dataframe for file

In [4]:
db.keys()

dict_keys(['Agricultural and Biological Sciences', 'All', 'Arts and Humanities', 'Biochemistry, Genetics and Molecular Biology', 'Business, Management and Accounting', 'Chemical Engineering', 'Chemistry', 'Computer Science', 'Decision Sciences', 'Dentistry', 'Earth and Planetary Sciences', 'Economics, Econometrics and Finance', 'Energy', 'Engineering', 'Environmental Science', 'Health Professions', 'Immunology and Microbiology', 'Materials Science', 'Mathematics', 'Medicine', 'Multidisciplinary', 'Neuroscience', 'Nursing', 'Pharmacology, Toxicology and Pharmaceutics', 'Physics and Astronomy', 'Psychology', 'Social Sciences', 'Veterinary'])

In [5]:
subject = list(db.keys())[0]
df_2017 = pd.DataFrame(db[subject][2017])
df_2017

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
4,Yes,No,No,AFG,Afghanistan,2017,36296.111,14485.543,12618.205,13,...,64.676697,NaN,NaN,NaN,NaN,91.89,NaN,NaN,NaN,NaN
8,Yes,No,No,ALB,Albania,2017,2884.169,3245.887,1892.473,44,...,73.224899,8.0,16.0,NaN,NaN,149.54,NaN,7150.0,1232.0,3497.0
12,Yes,No,No,DZA,Algeria,2017,41389.174,27028.330,22879.494,507,...,NaN,594.0,149.0,0.54243,819.34270,5201.61,41.86166,9582.0,8104.0,8882.0
16,No,No,No,ASM,American Samoa,2017,55.617,49.900,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,2017,76.997,58.882,NaN,3,...,NaN,6.0,NaN,NaN,NaN,2.01,NaN,2395.0,691.0,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.480,20557.694,16254.828,276,...,NaN,338.0,96.0,NaN,NaN,840.01,NaN,4025.0,18414.0,22439.0
704,Yes,No,No,VNM,Viet Nam,2017,94600.643,70883.488,58172.369,729,...,87.643600,4790.0,592.0,0.52674,707.74357,3343.23,71.49000,29463.0,55313.0,50733.0
887,Yes,No,No,YEM,Yemen,2017,27834.811,12954.157,10745.291,34,...,NaN,13.0,15.0,NaN,NaN,126.37,NaN,1772.0,2941.0,4713.0
894,Yes,No,No,ZMB,Zambia,2017,16853.608,8451.346,5725.647,105,...,73.935799,10.0,12.0,NaN,NaN,209.45,NaN,3413.0,1072.0,3147.0


In [6]:
df[(df['Year'] == 2017) & (df['Subject'] == 'Biochemistry, Genetics and Molecular Biology')]

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
Country code,,,,,,,,,,,,,,,,,,,,,
4,Yes,No,No,AFG,Afghanistan,2017,36296.111,14485.543,12618.205,7,...,64.676697,NaN,NaN,NaN,NaN,91.89,NaN,NaN,NaN,NaN
8,Yes,No,No,ALB,Albania,2017,2884.169,3245.887,1892.473,34,...,73.224899,8.0,16.0,NaN,NaN,149.54,NaN,7150.0,1232.0,3497.0
12,Yes,No,No,DZA,Algeria,2017,41389.174,27028.330,22879.494,339,...,NaN,594.0,149.0,0.54243,819.34270,5201.61,41.86166,9582.0,8104.0,8882.0
16,No,No,No,ASM,American Samoa,2017,55.617,49.900,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,2017,76.997,58.882,NaN,1,...,NaN,6.0,NaN,NaN,NaN,2.01,NaN,2395.0,691.0,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.480,20557.694,16254.828,127,...,NaN,338.0,96.0,NaN,NaN,840.01,NaN,4025.0,18414.0,22439.0
704,Yes,No,No,VNM,Viet Nam,2017,94600.643,70883.488,58172.369,527,...,87.643600,4790.0,592.0,0.52674,707.74357,3343.23,71.49000,29463.0,55313.0,50733.0
887,Yes,No,No,YEM,Yemen,2017,27834.811,12954.157,10745.291,51,...,NaN,13.0,15.0,NaN,NaN,126.37,NaN,1772.0,2941.0,4713.0


In [7]:
df

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
Country code,,,,,,,,,,,,,,,,,,,,,
8,Yes,No,No,ALB,Albania,1996,3098.699,2202.186,1553.434,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2094.0
12,Yes,No,No,DZA,Algeria,1996,29266.415,14872.253,11543.461,34,...,NaN,150.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,4214.0
16,No,No,No,ASM,American Samoa,1996,54.209,27.982,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,1996,64.363,25.571,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2449.0
32,Yes,No,No,ARG,Argentina,1996,35246.376,24259.564,18720.272,798,...,NaN,4012.0,1097.0,0.41749,NaN,NaN,NaN,NaN,NaN,47972.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2019,28515.829,21474.553,16060.699,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,Yes,No,No,VNM,Viet Nam,2019,96462.108,73651.220,60449.404,84,...,87.152702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
887,Yes,No,No,YEM,Yemen,2019,29161.922,14297.617,11664.767,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_1996_2017 = df[(df['Year'] <= 2017)]
df_1996_2017

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
Country code,,,,,,,,,,,,,,,,,,,,,
8,Yes,No,No,ALB,Albania,1996,3098.699,2202.186,1553.434,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2094.0
12,Yes,No,No,DZA,Algeria,1996,29266.415,14872.253,11543.461,34,...,NaN,150.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,4214.0
16,No,No,No,ASM,American Samoa,1996,54.209,27.982,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,1996,64.363,25.571,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2449.0
32,Yes,No,No,ARG,Argentina,1996,35246.376,24259.564,18720.272,798,...,NaN,4012.0,1097.0,0.41749,NaN,NaN,NaN,NaN,NaN,47972.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.480,20557.694,16254.828,46,...,NaN,338.0,96.0,NaN,NaN,840.01,NaN,4025.0,18414.0,22439.0
704,Yes,No,No,VNM,Viet Nam,2017,94600.643,70883.488,58172.369,55,...,87.643600,4790.0,592.0,0.52674,707.74357,3343.23,71.49,29463.0,55313.0,50733.0
887,Yes,No,No,YEM,Yemen,2017,27834.811,12954.157,10745.291,6,...,NaN,13.0,15.0,NaN,NaN,126.37,NaN,1772.0,2941.0,4713.0
